# How to execute and import a jupyter notebook?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?

import notebook?



In [1]:
// initialize
var path = require('path');
var fs = require('fs');

// provide a function for importing any notebook as a module and executing it
function getCellsOrDirectory(filename) {
    const fname = filename.replace('.ipynb', '');
    if (fs.existsSync(fname)
        && fs.statSync(fname).isDirectory()) {
        const files = fs.readdirSync(fname)
            .sort((a, b) => parseInt(a.replace(/[^0-9]/ig, '')) - parseInt(b.replace(/[^0-9]/ig, ''))) //sort numerically
            .filter(f => f.indexOf('.js') > -1);
        return files.map(f => fs.readFileSync(path.join(fname, f)).toString());
    }
    else {
        return getCells(filename, ['javascript', 'code'])
            .map((c, i) => ({
                fresher: c.source.join(''),
                id: path.basename(filename) + '[' + i + ']',
                filename: filename,
                runInNewContext: (ctx) => {
                    return runInNewContext(c.source.join(''), Object.assign(ctx || {}, {
                        __dirname: path.dirname(filename),
                        __filename: filename,
                    }), {filename: path.join(path.dirname(filename), path.basename(filename) + '[' + i + ']')}, false);
                }
            }));
    }
}

// output nothing here, cached version of this function is assigned below.
if (typeof imported !== 'object' || imported === null) {
    var imported = {};
}

// How to test if a notebook has already been imported?
function importNotebook(notebook, ctx = {}) {
    if (typeof notebook === 'undefined') {
        return Promise.resolve({});
    }
    var cells;
    // TODO: do Core notebooks synchronously?
    if(typeof notebook === 'string' && fs.existsSync(path.resolve(__dirname, notebook))) {
        // ONLY RETURN CACHE?
        if (typeof imported[notebook] !== 'undefined'
            && path.basename(notebook) === 'interpret questions.ipynb') {
            return imported[notebook];
        }
        cells = getCellsOrDirectory(path.resolve(path.resolve(__dirname, notebook)));
    } else {
        cells = interpret(notebook);
        if( typeof cells.fresher !== 'undefined') {
            imported[notebook] = cells.runInNewContext(ctx);
            return imported[notebook];
        }
    }
    
    imported[notebook] = cells.reduce((obj, c, i) => {
        var result = c.runInNewContext(Object.assign(ctx, obj));
        if(typeof result === 'object' && result !== null
           && typeof result[Object.keys(result)[0]] === 'function') {
            const func = result[Object.keys(result)[0]];
            obj[cells[i].id] = func;
            obj[func.name] = obj[cells[i].id];
        }
        if (typeof result === 'function') {
            obj[cells[i].id] = result;
            obj[result.name] = obj[cells[i].id];
        }
        obj[i] = result;
        Object.assign(ctx, obj);
        return obj;
    }, {});
    return imported[notebook];
}
if(typeof module.exports === 'undefined') {
    module.exports = {};
}
module.exports.import = importNotebook;
imported['import notebook.ipynb'] = module.exports;

// $$.done(); for our dumb parser



How to stream json using a {match} function?


In [ ]:
// initialize

var fs = require('fs');
var jp = require('jsonpath-plus');
var fileCache = {};

function streamJson(file, path, match = ((c) => true)) {
    const json = JSON.parse(fs.readFileSync(file).toString());
    const nodes = jp({json: Object.assign({}, json), path: '$..*', resultType: 'all', callback: n => {
        if(typeof n.value !== 'object' || n.value === null) {
            return;
        }
        return match({
            path: n.path.split(/[\\'\]\[\$]+/igm).slice(1),
            value: n.value
        });
    }});
//    return nodes
//        .filter(n => )
//        .forEach(n => );
}
if(typeof module.exports === 'undefined') {
    module.exports = {};
}
module.exports.streamJson = streamJson;


// $$.done() for our dumb parser



How to parse cells of {types} from a notebook?



In [ ]:
// initialize

// How to use getCells?
function getCells(notebook, types = ['*', 'code']) {
    console.log('reading notebook ' + notebook);
    var kernel = {};
    var cells = [];
    streamJson(notebook, [true, {emitPath: true}], (match) => {
        if (match.path[0] === 'metadata'
            && match.path[1] === 'kernelspec') {
            kernel = match.value;
        } else if (match.path[0] === 'cells'
            && types.indexOf(match.value.cell_type) > -1) {
            cells[cells.length] = match.value;
        }
    });
    if (types.indexOf(kernel.language) === -1
        && types.indexOf('*') === -1) {
        console.log('tried to import ' + types + JSON.stringify(kernel));
        return [];
    }
    return cells.map(c => Object.assign(c, {language: kernel.language}));
}
if(typeof module.exports === 'undefined') {
    module.exports = {};
}
module.exports.getCells = getCells;


// $$.done() for our dumb parser


How to run all promises sequentially?



In [ ]:
// initialize

function runAllPromises(promises) {
    return promises.reduce((promise, func) => {
        return promise.then(result => {
            return (typeof func == 'function'
                ? (new Promise(func)) : Promise.resolve(func))
                .then(Array.prototype.concat.bind(result));
        });
    }, Promise.resolve([]));
};
if(typeof module.exports === 'undefined') {
    module.exports = {};
}
module.exports.runAllPromises = runAllPromises;

// $$.done() for our dumb parser



How to turn jupyter notebooks in to modules?

# interpret(query)

This function has 4 modes:

```
search terms
search notebook.ipynb
search notebook.ipynb[0]
search notebook.ipynb[search terms]
```

Passing in a string returns the top match of a single cell, unless the string names a notebook in which case it returns and Array of all the cells in the notebook.
`[0]` returns the index of code cells, markdown cells are accumulated leading up to a code cell.
`[search terms]` searches the markdown within the specified notebook, it's garunteed to return a cell from the notebook or no match error.



In [1]:

// initialize
var path = require('path');

var fuseSearch;
var cacheCells;
function interpret(query) {
    if(typeof fuseSearch === 'undefined') {
        var interpreter = importNotebook('interpret all notebooks.ipynb')[0];
        fuseSearch = interpreter.fuseSearch;
        cacheCells = interpreter.cacheCells;
        // cache all notebooks
        interpreter.cacheAll(path.join(__dirname, '../'));
    }
    if(typeof query === 'undefined' || query === null || query === false) {
        // special lookup function for retrieving the entire index
        return cacheCells([], '');
    }
    const searches = (typeof query === 'string'
                      ? [query]
                      : query).map(q => fuseSearch(q));
    const fileMode = typeof query === 'string' && query.indexOf('.ipynb') > -1 && query.indexOf('[') === -1;
    // TODO: move this logic for creating the context in to the question interpreter so it don't have to loop so many times?
    const results = (fileMode ? searches[0] : searches).map(r => {
        if (typeof r === 'undefined') {
            return r;
        }
        if(typeof __webpack_require__ === 'function'
          && typeof getInjected !== 'undefined') {
            const runContext = getInjected()[r.id];
            if(runContext) {
                r.runInNewContext = runContext;
                return r;
            }
        }
        // must have a unique id for each unique cell so that
        // individual cells can serve as modules as well as notebooks
        // adding the cell id as a part of the filename
        r.runInNewContext = (ctx) => {
            return runInNewContext(
                r.fresher,
                Object.assign(ctx || {}, {
                    // TODO: don't use cache of parent modules that have includes that have changed
                    useCache: (mod) => r.mtime <= (mod.buildTime || 0),
                    __filename: r.filename
                }),
                {filename: path.join(path.dirname(path.resolve(r.filename)), r.id)}, false);
        };
        return r;
    });
    return typeof query === 'string' && !fileMode
        ? results[0]
        : results;
}

if(typeof module.exports === 'undefined') {
    module.exports = {};
}
module.exports.interpret = interpret;

// $$.done() for our dumb parser



[Function: interpret]

Import arbitrary code in to the current context?

Import arbitrary code in to a new context?

How does node module require work?

# runInNewContext(code, context)

Imports arbitrary code as a Node module.

## options

* useCache: true
* __filename: path or filename in errors
* 


TODO: webpack other languages and import?

TODO: use zeromq to import any language using other kernels?

TODO: use language server from VS Code?



In [ ]:
var path = require('path');
var vm = require('vm');

// initialize
function requireCode(code, pathToCode, tmpGlobal) {
    var Module = require('module').Module;
    var path = require('path');

    // must have a new name for every generation otherwise cache will be returned
    var filepath = path.resolve(process.cwd(), pathToCode);
    var dirname = path.dirname(filepath);

    var cachedModule = Module._cache[filepath];
    if (cachedModule && tmpGlobal.useCache !== false
        && (typeof tmpGlobal.useCache !== 'function'
            || tmpGlobal.useCache(cachedModule) !== false)) {
        tmpGlobal.module = cachedModule;
        return cachedModule.exports;
    }

    var mod = new Module(filepath, tmpGlobal.module);
    tmpGlobal.module = mod;
    Module._cache[filepath] = mod;

    mod.buildTime = (new Date()).getTime();
    mod.filename = filepath;
    mod.paths = Module._nodeModulePaths(dirname);
    
    // TODO: this is basically the magic of the 'rewire' module, reuse this?
    const validVars = Object.keys(tmpGlobal)
        .filter(k => k.match(/^[a-z_][a-z0-9_]*$/i))
        .join(',');

    console.log('compiling ' + pathToCode);
    
    mod._compile(`module.exports = (({${validVars}}) => {
${code}
})`, filepath);
    // assign the actual imports internally by calling the func export
    const result = mod.exports(tmpGlobal);
    tmpGlobal.exports = mod.exports;
    mod.loaded = true;
    return mod.exports;
}

// TODO: remove options
function runInNewContext(code, ctx, options) {
    const oldDir = process.cwd();
    const tmpGlobal = ctx || {};
    Object.assign(tmpGlobal, ctx);
    Object.assign(options || {}, { lineOffset: 1 });
    options.filename = options.filename || ctx.__filename;
    tmpGlobal.importer = require('../Core');
    // fixed reserved word usage
    //tmpGlobal.importer.import = tmpGlobal.importer['import' + 'Notebook'] = tmpGlobal.importer.import || tmpGlobal.importer['import' + 'Notebook'];
    // remove reference to Core and inject this context
    code = code.replace(/.*require\('\.\.\/Core'\).*;*\s*/igm, '');
    const result = requireCode(code, options.filename, tmpGlobal);
    return tmpGlobal.module.exports || result;
}

if(typeof module.exports === 'undefined') {
    module.exports = {};
}
module.exports.runInNewContext = runInNewContext;

// $$.done() for our dumb parser

